In [57]:
from faker import Faker
from faker.providers import DynamicProvider

In [9]:
import requests
import psycopg2
import pandas as pd
import numpy as np

In [64]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
cnxn = psycopg2.connect(host = "jobsearch-test.cz8su8kcabuv.us-east-2.rds.amazonaws.com",
                database = "postgres",
                user = "gdshoda",
                password = "p)aK1g3yW]1(6UbIeOGQN2j[u*B]",
                port = 5432
                       )     

In [42]:
fake_france = Faker('fr_FR')

In [43]:
fake_US = Faker('en_US')

In [13]:
cursor = cnxn.cursor()

In [24]:
cursor.execute("""SELECT n.nspname AS schema, t.typname AS type, e.enumlabel AS value
                FROM pg_type t 
                JOIN pg_enum e ON t.oid = e.enumtypid
                JOIN pg_namespace n ON n.oid = t.typnamespace
                ORDER BY schema, type, e.enumlabel;""")

In [25]:
enums = cursor.fetchall()

In [46]:
enums_df = pd.DataFrame(enums, columns = ['Acces', 'type', 'value'])

In [47]:
enums_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Acces   15 non-null     object
 1   type    15 non-null     object
 2   value   15 non-null     object
dtypes: object(3)
memory usage: 488.0+ bytes


In [50]:
enums_df['type'].unique()

array(['activity', 'correct_sizes', 'cover_letter'], dtype=object)

In [51]:
activities = enums_df.loc[enums_df['type'] == 'activity']

In [60]:
activities_provider = DynamicProvider(
    provider_name = "activity",
    elements = activities['type'].to_list()
)

In [32]:
cursor.execute('SELECT * FROM region;')

In [35]:
region_data = cursor.fetchall()

In [39]:
regions = pd.DataFrame(region_data, columns = ['id', 'iso', 'name','abbreviation'])

In [41]:
regions.head()

,id,iso,name,abbreviation
0,1,ALSACE,AA,FR
1,2,AQUITAINE,AQ,FR
2,3,AUVERGNE,AU,FR
3,4,BASSE-NORMANDIE,BN,FR
4,5,BOURGOGNE,BG,FR


# Loading Test Data into Dimension tables

## Countries table already has data

In [65]:
countries = pd.read_sql("SELECT * FROM country;", cnxn)

## Regions table already has data

In [66]:
regions = pd.read_sql("SELECT * FROM region;", cnxn)

## Company_size table already has data

In [67]:
company_size = pd.read_sql("SELECT * FROM company_size;", cnxn)

In [68]:
company_size.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      8 non-null      int64 
 1   size    8 non-null      object
dtypes: int64(1), object(1)
memory usage: 256.0+ bytes


## Loading Data into the Company Industries Table

In [76]:
industries = pd.read_sql("SELECT * FROM company_industry;", cnxn)

In [70]:
industries = industries = ["Finance", "Healthcare", "Technology", "Retail", "Marketing", "E-commerce", "Consulting", "Manufacturing", "Telecommunications","Education", "Energy", "Government"]

In [72]:
insert_query = "INSERT INTO company_industry (industry) VALUES (%s) ON CONFLICT (industry) DO NOTHING"

In [73]:
for industry in industries:
    cursor.execute(insert_query, (industry,))

In [74]:
cnxn.commit()

## Loading Data into the Languages table

In [81]:
languages_table = pd.read_sql("SELECT * FROM languages;", cnxn)

In [82]:
languages = ['English', 'French']

In [80]:
insert_language_query = "INSERT INTO languages (language) VALUES (%s) ON CONFLICT (language) DO NOTHING"

In [83]:
for language in languages:
    cursor.execute(insert_language_query, (language,))

In [84]:
cnxn.commit()

In [114]:
languages_table = pd.read_sql("SELECT * FROM languages;", cnxn)
languages_table

,id,language
0,1,English
1,2,French


## Loading Data into the Job Titles table

In [189]:
it_jobs = [
    "Software Developer", "Data Analyst", "Data Scientist", "System Administrator",
    "DevOps Engineer", "Network Engineer", "IT Support Specialist", "Web Developer",
    "Cloud Architect", "Cybersecurity Analyst", "Database Administrator", "Business Analyst"
]

In [190]:
insert_job_title_query = "INSERT INTO job_titles (title) VALUES (%s) ON CONFLICT (title) DO NOTHING"

In [198]:
titles_table = pd.read_sql("SELECT * FROM job_titles;", cnxn)

In [199]:
titles_table

,id,title
0,13,Software Developer
1,14,Data Analyst
2,15,Data Scientist
3,16,System Administrator
4,17,DevOps Engineer
5,18,Network Engineer
6,19,IT Support Specialist
7,20,Web Developer
8,21,Cloud Architect
9,22,Cybersecurity Analyst


In [194]:
for job in it_jobs:
    cursor.execute(insert_job_title_query, (job,))

In [195]:
cnxn.commit()

In [98]:
titles_table = pd.read_sql("SELECT * FROM job_titles;", cnxn)

## Loading data into the hiring_activity table

In [100]:
hiring_activity = pd.read_sql("SELECT * FROM hiring_activity;", cnxn)

In [101]:
hiring_activity

,id,activity


In [105]:
active = activities['value'].to_list()

In [106]:
active

['APPLICATION', 'ASSESSMENT', 'INTERVIEW', 'REJECTION']

In [119]:
insert_activity_query = "INSERT INTO hiring_activity (activity) VALUES (%s)"

In [120]:
for action in active:
    cursor.execute(insert_activity_query, (action,))

In [121]:
hiring_activity = pd.read_sql("SELECT * FROM hiring_activity;", cnxn)

In [122]:
hiring_activity

,id,activity
0,1,APPLICATION
1,2,ASSESSMENT
2,3,INTERVIEW
3,4,REJECTION


In [123]:
emails = pd.read_sql("SELECT * FROM email;", cnxn)

In [124]:
emails

,id,email
0,1,greg.shoda@gmail.com


# Creating a Fake Company Profile to insert into the DB

## Creating a Faker provider for company_size

In [152]:
pd.read_sql("SELECT * FROM company_size;", cnxn).id.to_list()

[1, 2, 3, 4, 5, 6, 7, 8]

In [153]:
company_size_provider = DynamicProvider(
    provider_name = "company_size",
    elements = pd.read_sql("SELECT * FROM company_size;", cnxn).id.to_list()
)

In [154]:
company_industry_provider_list = pd.read_sql("SELECT id FROM company_industry;", cnxn).id.to_list()

In [155]:
company_industry_provider_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [161]:
company_industry_provider = DynamicProvider(
    provider_name = "company_industry",
    elements = company_industry_provider_list
)

In [162]:
fake_US.add_provider(company_size_provider)

In [163]:
fake_US.add_provider(company_industry_provider)

In [164]:
company = fake_US.company()
company_city = fake_US.city()
company_state = fake_US.state().upper()
region_id = regions.loc[regions['name'] == company_state, 'id'].iloc[0]
company_size = fake_US.company_size()
company_industry = fake_US.company_industry()


In [168]:
print(f"Company: {company}, Company Headquerters: {company_city}, {company_state}, Company_Size: {company_size}, Company_Industry: {company_industry}")

Company: Adkins and Sons, Company Headquerters: Lake Jill, ALASKA, Company_Size: 8, Company_Industry: 5


In [136]:
regions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            111 non-null    int64 
 1   name          111 non-null    object
 2   abbreviation  111 non-null    object
 3   iso           111 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.6+ KB


In [ ]:
cursor.execute(

In [171]:
cursor.execute("""
                INSERT INTO locality (name, region_id)
                VALUES (%s, %s)
                RETURNING id;
                """, (company_city, int(region_id)))

In [172]:
locality_id = cursor.fetchone()[0]

In [174]:
print(company_size)

8


In [175]:
cursor.execute("""
                INSERT INTO company (name, hq_locality_id, company_size_id, company_ind_id)
                VALUES (%s, %s, %s, %s);
                """, (company, locality_id, company_size, company_industry))
                

In [176]:
cnxn.commit()

In [182]:
cursor.execute("""
                SELECT id
                FROM company;
                """, cnxn)

In [183]:
company_id = cursor.fetchone()[0]

In [184]:
posting = fake_US.url()
job_title_id = fake_US.random_int(min = , max = 12)
job_locality_id = 1


In [200]:
cursor.execute("""
                INSERT INTO job_details (posting, job_title_id, job_locality_id, company_id)
                VALUES (%s, %s, %s, %s)
                RETURNING id;
                """, (posting, 14, job_locality_id, company_id))

In [201]:
job_id = cursor.fetchone()[0]

In [202]:
job_id

3

In [203]:
cover_letter = "REQUIRED"
addt_ques = 0
references = 0
language_id = 1

In [207]:
cnxn.commit()

In [208]:
applications = pd.read_sql("SELECT * FROM application_details;", cnxn)

In [209]:
applications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            0 non-null      object
 1   cover_letter  0 non-null      object
 2   addt_ques     0 non-null      object
 3   reference     0 non-null      object
 4   expiration    0 non-null      object
 5   language_id   0 non-null      object
 6   account_id    0 non-null      object
 7   job_id        0 non-null      object
dtypes: object(8)
memory usage: 124.0+ bytes


In [215]:
cursor.execute("""
                ALTER TABLE application_details
                ALTER COLUMN account_id DROP NOT NULL;
                """, cnxn)

In [216]:
cnxn.commit()

In [217]:
cursor.execute("""
                INSERT INTO application_details (cover_letter, addt_ques, reference, language_id, job_id)
                VALUES (%s, %s::bit, %s::bit, %s, %s);
                """, (cover_letter, addt_ques, references, language_id, job_id))

In [218]:
cnxn.commit()